In [1]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math

# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
from time import sleep
import time

# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

# for문으로 크롤링 

In [51]:
# 검색어 입력
search = '삼성전자'

# 크롬 옵션
options = webdriver.ChromeOptions()

# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1000,1200") # window-size -> 기본 : 1920,1080

# import chromedriver_autoinstaller

# chrome_path = chromedriver_autoinstaller.install()
# driver = webdriver.Chrome(chrome_path, options=options)
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

time.sleep(0.01)
driver.get("https://www.naver.com") # -> 네이버 크롬 창이 뜬다.

# 네이버 검색어 입력 후 검색
element = driver.find_element("name", "query") 
element.clear()  # 혹시 검색창에 존재하는 텍스트 제거

element.send_keys(search)  # 검색창에 검색어 전달
element.submit()  # 검색 클릭
time.sleep(0.01)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [/Users/krc/.wdm/drivers/chromedriver/mac64_m1/103.0.5060.134/chromedriver] found in cache


In [52]:
# 뉴스 클릭
driver.find_element("link text", "뉴스").click() 
time.sleep(0.01)

In [53]:
# '오래된 순' 클릭
driver.find_element("link text", "오래된순").click()  
time.sleep(0.01)

In [70]:
# 우선 수기로 옵션 누르고 조회 기간을 설정해줌

In [54]:
# 뉴스 데이터 수집
news_list = []
date_list = []
title_list = []
MAX_ITERATION = 380

for _ in range(120):
    time.sleep(1)

    # 기간 변경해 주기
    if len(date_list):

        for index in range(1, 10):
            splited_date = date_list[-index].split('.')
            if (len(splited_date[0]) == 4) and (len(splited_date[1]) == 2) and (len(splited_date[2]) == 2):
                break 

        # splited_date = date_list[-1].split('.')
        year = int(splited_date[0]) - 1990 + 1
        month = int(splited_date[1])
        day = int(splited_date[2])

        # 기간을 클릭 해서 입력창 활성화  
        time.sleep(4)   
        driver.find_element(by=By.CSS_SELECTOR, value='#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.option > a.txt.txt_option._calendar_select_trigger').click()
        time.sleep(2)

        ## 시작 일자 조정 ##
        
        # 년도 
        # 2016년 CSS_SELECTOR : snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(1) > div > div > div > ul > li:nth-child(27)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(1) > div > div > div > ul > li:nth-child({year})').click()
        time.sleep(1)
        
        # 월
        # 1월 CSS_SELECTOR : snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(2) > div > div > div > ul > li:nth-child(1)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(2) > div > div > div > ul > li:nth-child({month})').click()
        time.sleep(1)
        
        # 일
        # 1일 CSS_SELECTOR : #snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(3) > div > div > div > ul > li:nth-child(1)
        driver.find_element(by=By.CSS_SELECTOR, value=f'#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.select_wrap._root > div:nth-child(3) > div > div > div > ul > li:nth-child({day})').click()
        time.sleep(1)
        
        # "적용" 버튼 클릭
        driver.find_element(by=By.CSS_SELECTOR, value='#snb > div.api_group_option_sort._search_option_detail_wrap > ul > li.bx.term > div > div.api_select_option.type_calendar._calendar_select_layer > div.btn_area > button').click()
        time.sleep(2)
        
    for i in range(MAX_ITERATION):

        # 언론사
        newses = driver.find_elements('css selector', 'a.info.press')
        temp_list = []
        for news in newses:
            temp_list.append(news.text)
            temp_list = [x for x in temp_list ]
        news_list = news_list + temp_list 
        
        # 날짜
        dates = driver.find_elements('css selector', 'span.info')
        temp_list = []
        for date in dates:
            temp_list.append(date.text)
            temp_list = [x for x in temp_list ]
        date_list = date_list + temp_list 
        
        # 제목
        titles = driver.find_elements('css selector', 'a.news_tit')
        temp_list = []
        for title in titles:
            temp_list.append(title.text)
            temp_list = [x for x in temp_list ]
        title_list = title_list + temp_list 
        
        # 다음페이지 클릭
        driver.find_element('css selector', 'a.btn_next').click()
        time.sleep(0.001)

In [55]:
print(len(title_list))
print(news_list[-2:])
print(date_list[-2:])
print(title_list[-2:])

456000
['아주경제', '아시아타임즈']
['2020.10.27.', '2020.10.27.']
["[이건희 회장 별세] 사흘째 재계 '큰 별' 기렸다...28일 '마지막 출근길' 오를...", "[마감] LG화학, 국민연금 배터리 분할 '반대'...에스와이, 솔라루프 기대감 '..."]


In [56]:
# df로 만들기
df = pd.DataFrame(list(zip(news_list, date_list, title_list)), columns = ['언론사','날짜', '제목'])
df

,언론사,날짜,제목
0,글로벌이코노믹,2019.03.27.,"[뉴욕증시] 다우지수 흔들, R의 공포 금리역전 갈수록 더 심각… 원달러환율 ..."
1,코리아중앙데일리,2019.03.27.,A hamstrung tech industry (KOR)
2,뉴스토마토,2019.03.27.,"(ELF 재테크)①ELS 닮은꼴…""급락장은 없다""에 베팅"
3,아주경제,2019.03.27.,전날 외국인 코스피 순매수 상위종목에 삼성전자·삼성물산
4,아주경제,2019.03.27.,[아주증시포커스] 상장사·회계사 모두 부담스러운 새 외감법
...,...,...,...
455938,뉴스핌,2020.09.17.,고 이건희 회장 빈소 나서는 문성현 경제사회노동위원회 위원장
455939,한국경제언론사 선정,14면 TOP,경주 양성자가속기 빔 이용 경쟁률 '역대 최고'
455940,YTN,2020.09.17.,인재경영
455941,아주경제,14면 1단,"2020 국정감사 기업 이슈 재조명...""로드숍 불공정 운영, 하도급법 위반 및 기..."


In [57]:
# csv로 뽑기
df.to_csv('2019-03-17~2020-09-17.csv')

In [58]:
driver.close()